In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import missingno as msno
import seaborn as sns
import scipy.stats as st
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xg
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
from eli5.sklearn import PermutationImportance
from eli5 import show_weights
from sklearn.tree import DecisionTreeClassifier
from bayes_opt import BayesianOptimization

In [2]:
#Загрузим данные из файла.
df = pd.read_excel('/home/rik/Рабочий стол/МИФИ/учеба/мо/curse/data.xlsx', index_col=0)
df.head()

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print("train.shape = {} rows, {} cols".format(*df.shape))

train.shape = 1001 rows, 213 cols


# Предобработка данных.

In [4]:
def transform(df): 
    discrete_feature = [feature for feature in df.columns if len(df[feature].unique())<25]
    count_discrete_feature = {}
    for i in tqdm(discrete_feature):
        cnt = 0
        for j in df[i]:
            if j != 0:
                cnt += 1
        count_discrete_feature[f'{i}'] = cnt
    count_discrete_feature = pd.Series(count_discrete_feature)
    df = df.drop(columns=count_discrete_feature[count_discrete_feature == 0].index.to_list())
    df = df.dropna()
    feature_del = ['BertzCT', 'Chi0', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v',
     'ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan3', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Kappa1', 'Kappa2',
     'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MolMR', 'MolWt', 'NHOHCount', 'NumAromaticCarbocycles',
     'NumHAcceptors', 'NumHeteroatoms', 'NumSaturatedCarbocycles', 'NumValenceElectrons', 'SMR_VSA4', 'SMR_VSA9',
     'SlogP_VSA6', 'TPSA', 'VSA_EState2', 'VSA_EState3', 'VSA_EState6', 'fr_Al_OH_noTert', 'fr_COO', 'fr_COO2',
     'fr_C_O_noCOO', 'fr_Nhpyrrole', 'fr_benzene', 'fr_nitro_arom_nonortho', 'fr_phenol', 'fr_phenol_noOrthoHbond',
    'MinAbsPartialCharge', 'Chi2n']
    df = df.drop(columns=feature_del)
    df =  df[df['SI'] < 250]
    
    return df

In [5]:
df = transform(df)

100%|███████████████████████████████████████| 110/110 [00:00<00:00, 6459.19it/s]


In [6]:
df['SI'] = df['SI'].apply(lambda x: 0 if x < 8 else 1)
df['SI'].head()

0    1
1    0
2    0
3    1
4    0
Name: SI, dtype: int64

## Обучение нескольких моделей, их сравнение. 

In [7]:
target = df['SI']
data = df.drop(['CC50, mM','IC50, mM','SI'], axis=1)

In [8]:
x_train, x_valid = train_test_split(
    data, train_size=0.8, random_state=1
)
y_train, y_valid = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))

x_train.shape = 776 rows, 144 cols
x_valid.shape = 195 rows, 144 cols


In [9]:
# Создадим модель дерева решений
tree = DecisionTreeClassifier(random_state=27)
# Создадим модель CatBoost
model = CatBoostClassifier(random_state=27)
# Создадим модель lightgbm
model_LGBM = LGBMClassifier(random_state=27)
# Создадим модель xgboost
xgb_r = xg.XGBClassifier() 
# обучение моделей
tree.fit(x_train, y_train)
model.fit(x_train, y_train, verbose=0)
model_LGBM.fit(x_train, y_train, verbose=0)
xgb_r.fit(x_train, y_train, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_tree_train = tree.predict(x_train)
y_pred_сat_Boost_train = model.predict(x_train)
y_pred_LGBM_train = model_LGBM.predict(x_train)
y_pred_xgb_train = xgb_r.predict(x_train)
# предсказание ответов для тестовой выборки
y_pred_tree = tree.predict(x_valid)
y_pred_сat_Boost = model.predict(x_valid)
y_pred_LGBM = model_LGBM.predict(x_valid)
y_pred_xgb = xgb_r.predict(x_valid)

print("Train tree f1_score: ", f1_score(y_train, y_pred_tree_train))
print("Train tree accuracy_score: ", accuracy_score(y_train, y_pred_tree_train))
print("Validation tree f1_score: ", f1_score(y_valid, y_pred_tree))
print("Validation tree accuracy_score: ", accuracy_score(y_valid, y_pred_tree))
print('*' * 50)
print("Train сat_Boost f1_score: ", f1_score(y_train, y_pred_сat_Boost_train))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train, y_pred_сat_Boost_train))
print("Validation сat_Boost f1_score: ", f1_score(y_valid, y_pred_сat_Boost))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid, y_pred_сat_Boost))
print('*' * 50)
print("Train LGBM f1_score: ", f1_score(y_train, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid, y_pred_LGBM))
print('*' * 50)
print("Train xgb f1_score: ", f1_score(y_train, y_pred_xgb_train))
print("Train xgb accuracy_score: ", accuracy_score(y_train, y_pred_xgb_train))
print("Validation xgb f1_score: ", f1_score(y_valid, y_pred_xgb))
print("Validation xgb accuracy_score: ", accuracy_score(y_valid, y_pred_xgb))

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Train tree f1_score:  0.9130434782608695
Train tree accuracy_score:  0.9432989690721649
Validation tree f1_score:  0.44827586206896547
Validation tree accuracy_score:  0.6717948717948717
**************************************************
Train сat_Boost f1_score:  0.842323651452282
Train сat_Boost accuracy_score:  0.9020618556701031
Validation сat_Boost f1_score:  0.5321100917431192
Validation сat_Boost accuracy_score:  0.7384615384615385
**************************************************
Train LGBM f1_score:  0.915057915057915
Train LGBM accuracy_score:  0.9432989690721649
Validation LGBM f1_score:  0.5535714285714285
Validation LGBM accuracy_score:  0.7435897435897436
**************************************************
Train xgb f1_score:  0.9153846153846155
Train xgb accuracy_score:  0.9432989690721649
Validation xgb f1_score:  0.543859649122807
Validation xgb accuracy_score:  0.7333333333333333


Из приведенных данных выше мы видим низкое и среднее качество моделей, а также переобучение. Наилучшие результаты показали модели сat_Boost, LGBM, xgb. С ними и продолжим работать.

# Подбор признаков, их анализ и оценка важности.

**CatBoost**

In [10]:
col = x_train.columns.tolist()
estimator_cat_boost = PermutationImportance(
    model, scoring= "f1", random_state=27
)
importance_cat_boost = estimator_cat_boost.fit(x_train, y_train)

In [11]:
show_weights(importance_cat_boost, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0231 ± 0.0134,MinAbsEStateIndex
0.0136 ± 0.0051,VSA_EState4
0.0130 ± 0.0028,EState_VSA7
0.0116 ± 0.0117,PEOE_VSA6
0.0113 ± 0.0063,PEOE_VSA8
0.0091 ± 0.0068,FpDensityMorgan2
0.0087 ± 0.0069,EState_VSA3
0.0081 ± 0.0085,Ipc
0.0080 ± 0.0054,SMR_VSA5
0.0079 ± 0.0065,BCUT2D_MRHI


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [12]:
feature_cat_boost = [
'MinAbsEStateIndex','VSA_EState4','EState_VSA7','PEOE_VSA6','PEOE_VSA8','FpDensityMorgan2','EState_VSA3','Ipc',
'SMR_VSA5','BCUT2D_MRHI','MolLogP','SlogP_VSA4','VSA_EState8','EState_VSA6','BCUT2D_CHGHI','PEOE_VSA11','MaxEStateIndex',
'Chi0n','qed','VSA_EState10','EState_VSA4','MinPartialCharge','EState_VSA2','fr_aryl_methyl','SlogP_VSA5',
'EState_VSA8','MinEStateIndex','PEOE_VSA12','PEOE_VSA9','PEOE_VSA2','SlogP_VSA2','PEOE_VSA1','EState_VSA9',
'SMR_VSA7','SlogP_VSA10','fr_ester','NumRotatableBonds','MaxPartialCharge','fr_bicyclic','PEOE_VSA14','BCUT2D_LOGPLOW',
'PEOE_VSA7','VSA_EState7','SMR_VSA10','BCUT2D_MWHI','PEOE_VSA3','SlogP_VSA12','AvgIpc','BCUT2D_MRLOW','RingCount',
'PEOE_VSA4','fr_priamide','PEOE_VSA10','fr_ketone','fr_C_O','NumAliphaticHeterocycles','EState_VSA10','NumAliphaticRings',
'SMR_VSA1','SlogP_VSA7','fr_pyridine','SMR_VSA3','HallKierAlpha','fr_aniline','PEOE_VSA5','SlogP_VSA8','fr_para_hydroxylation',
'fr_methoxy','NumAromaticRings','NOCount','NumAliphaticCarbocycles','SPS','fr_unbrch_alkane','VSA_EState9',
'fr_halogen','fr_imidazole','fr_Ar_NH','fr_quatN','EState_VSA5','SlogP_VSA1','fr_thiazole','fr_nitro','fr_allylic_oxid',
'fr_amide']

In [13]:
target = df['SI']
data_cb = data[feature_cat_boost]

In [14]:
x_train_cb, x_valid_cb = train_test_split(
    data_cb, train_size=0.8, random_state=1
)
y_train_cb, y_valid_cb = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_cb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_cb.shape))

x_train.shape = 776 rows, 84 cols
x_valid.shape = 195 rows, 84 cols


In [15]:
# Создадим модель CatBoost
model = CatBoostClassifier(random_state=27)
# обучение моделей
model.fit(x_train_cb, y_train_cb, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_сat_Boost_train = model.predict(x_train_cb)
# предсказание ответов для тестовой выборки
y_pred_сat_Boost = model.predict(x_valid_cb)


print("Train сat_Boost f1_score: ", f1_score(y_train_cb, y_pred_сat_Boost_train))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train_cb, y_pred_сat_Boost_train))
print("Validation сat_Boost f1_score: ", f1_score(y_valid_cb, y_pred_сat_Boost))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid_cb, y_pred_сat_Boost))


Train сat_Boost f1_score:  0.8594704684317719
Train сat_Boost accuracy_score:  0.9110824742268041
Validation сat_Boost f1_score:  0.5504587155963303
Validation сat_Boost accuracy_score:  0.7487179487179487


Отобрав признаки мы немного улучшили метрики для модели CatBoost на вадидации f1_score:  0.55, accuracy_score:  0.75.

**LGBM**

In [16]:
col = x_train.columns.tolist()
estimator_LGBM = PermutationImportance(
    model_LGBM, scoring= "f1", random_state=27
)
importance_LGBM = estimator_LGBM.fit(x_train, y_train)


In [17]:
show_weights(importance_LGBM, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0216 ± 0.0155,MaxEStateIndex
0.0092 ± 0.0119,MinPartialCharge
0.0070 ± 0.0086,MolLogP
0.0070 ± 0.0087,FractionCSP3
0.0067 ± 0.0082,BCUT2D_CHGLO
0.0059 ± 0.0186,PEOE_VSA8
0.0046 ± 0.0082,VSA_EState4
0.0037 ± 0.0151,HallKierAlpha
0.0033 ± 0.0102,BCUT2D_MRLOW
0.0030 ± 0.0118,qed


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [18]:
feature_LGBM = ['MaxEStateIndex','MinPartialCharge','MolLogP','FractionCSP3','BCUT2D_CHGLO','VSA_EState4',
'HallKierAlpha','BCUT2D_MRLOW','qed','BCUT2D_LOGPHI','EState_VSA9','MinEStateIndex','NumAromaticRings',
'AvgIpc','SPS','RingCount','PEOE_VSA2','PEOE_VSA5','BCUT2D_MWLOW','EState_VSA2','BCUT2D_MWHI','MinAbsEStateIndex',
'SlogP_VSA1','EState_VSA4','fr_Ar_N','PEOE_VSA11','BCUT2D_MRHI','NumSaturatedRings','EState_VSA3','PEOE_VSA9',
'EState_VSA5','PEOE_VSA3','BalabanJ','PEOE_VSA1','fr_ester','NumRotatableBonds','SMR_VSA1','SMR_VSA10','VSA_EState1',
'VSA_EState10','EState_VSA8','PEOE_VSA12','VSA_EState7','EState_VSA10','BCUT2D_LOGPLOW','PEOE_VSA14','NumHDonors',
'NumAliphaticHeterocycles']

In [19]:
target_LGBM = df['SI']
data_LGBM = data[feature_LGBM]

In [20]:
x_train_LGBM, x_valid_LGBM = train_test_split(
    data_LGBM, train_size=0.8, random_state=1
)
y_train_LGBM, y_valid_LGBM = train_test_split(
    target_LGBM, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_LGBM.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_LGBM.shape))

x_train.shape = 776 rows, 48 cols
x_valid.shape = 195 rows, 48 cols


In [21]:
model_LGBM = LGBMClassifier(random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM)
# предсказание ответов для тренеровочной выборки
y_pred_LGBM_train = model_LGBM.predict(x_train_LGBM)
# предсказание ответов для тестовой выборки
y_pred_LGBM = model_LGBM.predict(x_valid_LGBM)

print("Train LGBM f1_score: ", f1_score(y_train_LGBM, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train_LGBM, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid_LGBM, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid_LGBM, y_pred_LGBM))

Train LGBM f1_score:  0.914396887159533
Train LGBM accuracy_score:  0.9432989690721649
Validation LGBM f1_score:  0.5688073394495413
Validation LGBM accuracy_score:  0.7589743589743589


Отобрав признаки мы улучшили метрики для модели LGBMClassifier на вадидации f1_score:  0.57, accuracy_score:  0.76.

**xgboost**

In [22]:
col = x_train.columns.tolist()
estimator_xgb = PermutationImportance(
    xgb_r, scoring= "neg_mean_squared_error", random_state=27
)
importance_xgb = estimator_xgb.fit(x_train, y_train)

In [23]:
show_weights(importance_xgb, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0085 ± 0.0048,BCUT2D_MRLOW
0.0077 ± 0.0082,BCUT2D_LOGPHI
0.0062 ± 0.0044,MinPartialCharge
0.0059 ± 0.0035,MolLogP
0.0046 ± 0.0128,SMR_VSA7
0.0044 ± 0.0116,MinAbsEStateIndex
0.0036 ± 0.0038,VSA_EState4
0.0036 ± 0.0030,PEOE_VSA9
0.0036 ± 0.0044,PEOE_VSA8
0.0034 ± 0.0042,EState_VSA2


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [24]:
feature_xgb = ['BCUT2D_MRLOW','BCUT2D_LOGPHI','MinPartialCharge','MolLogP','MinAbsEStateIndex','VSA_EState4',
'PEOE_VSA9','PEOE_VSA8','EState_VSA2','MinEStateIndex','AvgIpc','EState_VSA4','PEOE_VSA3','VSA_EState8',
'MaxEStateIndex','FractionCSP3','BCUT2D_MRHI','SMR_VSA10','PEOE_VSA6','qed','Chi0n','EState_VSA7','SPS','PEOE_VSA10',
'RingCount','EState_VSA3','BCUT2D_MWLOW','HallKierAlpha','fr_C_O','fr_NH0','SlogP_VSA1','VSA_EState10','BCUT2D_CHGLO',
'NOCount','SlogP_VSA4','fr_NH1','BCUT2D_CHGHI','VSA_EState9','PEOE_VSA5','BCUT2D_MWHI','NumSaturatedHeterocycles',
'NumAliphaticCarbocycles','EState_VSA9','Ipc','VSA_EState1','EState_VSA10','VSA_EState7','SMR_VSA1','NumRotatableBonds',
'BCUT2D_LOGPLOW','SlogP_VSA3','FpDensityMorgan2']

In [25]:
target_xgb = df['SI']
data_xgb = data[feature_xgb]

In [26]:
x_train_xgb, x_valid_xgb = train_test_split(
    data_xgb, train_size=0.8, random_state=1
)
y_train_xgb, y_valid_xgb = train_test_split(
    target_xgb, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_xgb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_xgb.shape))

x_train.shape = 776 rows, 52 cols
x_valid.shape = 195 rows, 52 cols


In [27]:
model_xgb = xg.XGBClassifier(random_state=27)
model_xgb.fit(x_train_xgb, y_train_xgb, verbose=100)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=27, ...)

In [28]:
y_pred_xgb = model_xgb.predict(x_train_xgb)
y_val_xgb = model_xgb.predict(x_valid_xgb)
print("Train f1_score: ", f1_score(y_train_xgb, y_pred_xgb))
print("Train accuracy_score: ", accuracy_score(y_train_xgb, y_pred_xgb))
print("Validation f1_score: ", f1_score(y_valid_xgb, y_val_xgb))
print("Validation accuracy_score: ", accuracy_score(y_valid_xgb, y_val_xgb))

Train f1_score:  0.9140625
Train accuracy_score:  0.9432989690721649
Validation f1_score:  0.5913043478260869
Validation accuracy_score:  0.7589743589743589


Отобрав признаки мы улучшили метрики для модели xgboost на вадидации f1_score:  0.59, accuracy_score:  0.76.

# Подбор гиперпараметров

**CatBoost**

In [29]:
def catboost_cv(depth, learning_rate,n_estimators):
    depth = int(depth)
    n_estimators = int(n_estimators)
    model = CatBoostClassifier(
        depth=depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        eval_metric  = 'F1',
        random_seed=27
    )
    model.fit(x_train_cb, y_train_cb, verbose=100)

    return f1_score(y_valid_cb, model.predict(x_valid_cb))

In [30]:
param_space = {
    'depth': (2, 10),             
    'learning_rate': (0.001, 0.3),
    'n_estimators': (50, 1000)
}

bayesian_opt = BayesianOptimization(
    f=catboost_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=20)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   |   depth   | learni... | n_esti... |
-------------------------------------------------------------
0:	learn: 0.3860590	total: 19.6ms	remaining: 14.6s
100:	learn: 0.8986083	total: 491ms	remaining: 3.15s
200:	learn: 0.9147287	total: 899ms	remaining: 2.45s
300:	learn: 0.9147287	total: 1.31s	remaining: 1.94s
400:	learn: 0.9160305	total: 1.78s	remaining: 1.54s
500:	learn: 0.9157088	total: 2.25s	remaining: 1.11s
600:	learn: 0.9147287	total: 2.78s	remaining: 681ms
700:	learn: 0.9147287	total: 3.2s	remaining: 215ms
747:	learn: 0.9163498	total: 3.39s	remaining: 0us
| 1         | 0.5263    | 5.406     | 0.2446    | 748.6     |
0:	learn: 0.5058548	total: 25.4ms	remaining: 24.8s
100:	learn: 0.9139579	total: 2.54s	remaining: 22.2s
200:	learn: 0.9147287	total: 5.01s	remaining: 19.4s
300:	learn: 0.9147287	total: 7.33s	remaining: 16.5s
400:	learn: 0.9150579	total: 9.66s	remaining: 13.9s
500:	learn: 0.9143969	total: 12.1s	remaining: 11.5s
600:	learn: 0.9147287	total: 14.4s	rema

| 21        | 0.5424    | 2.697     | 0.1964    | 340.2     |
0:	learn: 0.3860590	total: 7.64ms	remaining: 6.61s
100:	learn: 0.9129594	total: 655ms	remaining: 4.96s
200:	learn: 0.9160305	total: 1.3s	remaining: 4.3s
300:	learn: 0.9157088	total: 1.95s	remaining: 3.65s
400:	learn: 0.9153846	total: 2.6s	remaining: 3.01s
500:	learn: 0.9160305	total: 3.25s	remaining: 2.36s
600:	learn: 0.9150579	total: 3.95s	remaining: 1.73s
700:	learn: 0.9143969	total: 4.6s	remaining: 1.07s
800:	learn: 0.9147287	total: 5.26s	remaining: 420ms
864:	learn: 0.9160305	total: 5.7s	remaining: 0us
| 22        | 0.5133    | 6.781     | 0.172     | 865.4     |
0:	learn: 0.4263158	total: 12.9ms	remaining: 2.91s
100:	learn: 0.9153846	total: 1.17s	remaining: 1.45s
200:	learn: 0.9140625	total: 2.33s	remaining: 290ms
225:	learn: 0.9153846	total: 2.63s	remaining: 0us
| 23        | 0.4956    | 7.484     | 0.2354    | 226.6     |
0:	learn: 0.0000000	total: 3.06ms	remaining: 1.03s
100:	learn: 0.2687500	total: 141ms	remaining: 

In [31]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'l2_leaf_reg'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'depth': 9, 'learning_rate': 0.2005611810761974, 'n_estimators': 227.93938171457134}
Best R-squared Score: 0.5841


In [32]:
params = {
    'depth': 9,             
    'learning_rate': 0.2005611810761974, 
    'n_estimators': 228
}
model = CatBoostClassifier(**params, random_state=27)
model.fit(x_train_cb, y_train_cb, verbose=100)
y_pred_cb = model.predict(x_train_cb)
y_val_cb = model.predict(x_valid_cb)
print("Train сat_Boost f1_score: ", f1_score(y_train_cb, y_pred_cb))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train_cb, y_pred_cb))
print("Validation сat_Boost f1_score: ", f1_score(y_valid_cb, y_val_cb))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid_cb, y_val_cb))

0:	learn: 0.6237191	total: 53.6ms	remaining: 12.2s
100:	learn: 0.1201141	total: 4.76s	remaining: 5.98s
200:	learn: 0.1012288	total: 9.48s	remaining: 1.27s
227:	learn: 0.0995476	total: 10.8s	remaining: 0us
Train сat_Boost f1_score:  0.9153846153846155
Train сat_Boost accuracy_score:  0.9432989690721649
Validation сat_Boost f1_score:  0.584070796460177
Validation сat_Boost accuracy_score:  0.7589743589743589


После подбора гиперпараметров мы улучшили метрики модели CatBoost на вадидации f1_score:  0.58, accuracy_score:  0.76.

**LGBM**

In [33]:
def LGBM_cv(max_depth, learning_rate, n_estimators):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = LGBMClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,          
        eval_metric  = 'F1',
        random_seed=27
    )
    model.fit(x_train_LGBM, y_train_LGBM, eval_set = (x_valid_LGBM, y_valid_LGBM))
    return f1_score(y_valid_LGBM, model.predict(x_valid_LGBM))

In [34]:
param_space = {
    'max_depth': (2, 10),            
    'learning_rate': (0.001, 0.3),
    'n_estimators': (50, 1000)
}

bayesian_opt = BayesianOptimization(
    f=LGBM_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=25)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's binary_logloss: 0.608123
[2]	valid_0's binary_logloss: 0.598006
[3]	valid_0's binary_logloss: 0.588786
[4]	valid_0's binary_logloss: 0.58126
[5]	valid_0's binary_logloss: 0.576983
[6]	valid_0's binary_logloss: 0.573339
[7]	valid_0's binary_logloss: 0.56972
[8]	valid_0's binary_logloss: 0.570631
[9]	valid_0's binary_logloss: 0.57009
[10]	valid_0's binary_logloss: 0.575064
[11]	valid_0's binary_logloss: 0.573994
[12]	valid_0's binary_logloss: 0.572611
[13]	valid_0's binary_logloss: 0.57134
[14]	valid_0's binary_logloss: 0.573504
[15]	valid_0's binary_logloss: 0.578493
[16]	valid_0's binary_logloss: 0.583252
[17]	valid_0's binary_logloss: 0.58023
[18]	valid_0's binary_logloss: 0.582279
[19]	valid_0's binary_logloss: 0.579354
[20]	valid_0's binary_logloss: 0.577693
[21]	valid_0's binary_logloss: 0.57703

[274]	valid_0's binary_logloss: 1.15007
[275]	valid_0's binary_logloss: 1.15086
[276]	valid_0's binary_logloss: 1.15199
[277]	valid_0's binary_logloss: 1.15264
[278]	valid_0's binary_logloss: 1.15261
[279]	valid_0's binary_logloss: 1.15304
[280]	valid_0's binary_logloss: 1.15414
[281]	valid_0's binary_logloss: 1.15936
[282]	valid_0's binary_logloss: 1.16175
[283]	valid_0's binary_logloss: 1.16511
[284]	valid_0's binary_logloss: 1.16826
[285]	valid_0's binary_logloss: 1.16807
[286]	valid_0's binary_logloss: 1.16913
[287]	valid_0's binary_logloss: 1.1708
[288]	valid_0's binary_logloss: 1.17245
[289]	valid_0's binary_logloss: 1.17373
[290]	valid_0's binary_logloss: 1.17431
[291]	valid_0's binary_logloss: 1.17692
[292]	valid_0's binary_logloss: 1.17882
[293]	valid_0's binary_logloss: 1.17841
[294]	valid_0's binary_logloss: 1.17924
[295]	valid_0's binary_logloss: 1.18111
[296]	valid_0's binary_logloss: 1.18378
[297]	valid_0's binary_logloss: 1.18762
[298]	valid_0's binary_logloss: 1.18738
[

[574]	valid_0's binary_logloss: 1.44456
[575]	valid_0's binary_logloss: 1.44511
[576]	valid_0's binary_logloss: 1.44616
[577]	valid_0's binary_logloss: 1.44631
[578]	valid_0's binary_logloss: 1.4451
[579]	valid_0's binary_logloss: 1.44532
[580]	valid_0's binary_logloss: 1.44522
[581]	valid_0's binary_logloss: 1.44673
[582]	valid_0's binary_logloss: 1.44702
[583]	valid_0's binary_logloss: 1.44839
[584]	valid_0's binary_logloss: 1.44878
[585]	valid_0's binary_logloss: 1.44925
[586]	valid_0's binary_logloss: 1.45014
[587]	valid_0's binary_logloss: 1.45072
[588]	valid_0's binary_logloss: 1.45145
[589]	valid_0's binary_logloss: 1.45147
[590]	valid_0's binary_logloss: 1.45264
[591]	valid_0's binary_logloss: 1.45163
[592]	valid_0's binary_logloss: 1.45152
[593]	valid_0's binary_logloss: 1.45386
[594]	valid_0's binary_logloss: 1.45452
[595]	valid_0's binary_logloss: 1.45651
[596]	valid_0's binary_logloss: 1.45681
[597]	valid_0's binary_logloss: 1.45777
[598]	valid_0's binary_logloss: 1.45893
[

[122]	valid_0's binary_logloss: 0.926972
[123]	valid_0's binary_logloss: 0.932354
[124]	valid_0's binary_logloss: 0.935301
[125]	valid_0's binary_logloss: 0.940111
[126]	valid_0's binary_logloss: 0.943541
[127]	valid_0's binary_logloss: 0.944808
[128]	valid_0's binary_logloss: 0.945579
[129]	valid_0's binary_logloss: 0.9504
[130]	valid_0's binary_logloss: 0.953028
[131]	valid_0's binary_logloss: 0.955523
[132]	valid_0's binary_logloss: 0.959538
[133]	valid_0's binary_logloss: 0.961745
[134]	valid_0's binary_logloss: 0.967126
[135]	valid_0's binary_logloss: 0.969749
[136]	valid_0's binary_logloss: 0.974426
[137]	valid_0's binary_logloss: 0.975812
[138]	valid_0's binary_logloss: 0.97839
[139]	valid_0's binary_logloss: 0.984873
[140]	valid_0's binary_logloss: 0.983015
[141]	valid_0's binary_logloss: 0.987047
[142]	valid_0's binary_logloss: 0.988474
[143]	valid_0's binary_logloss: 0.992185
[144]	valid_0's binary_logloss: 0.994093
[145]	valid_0's binary_logloss: 0.997141
[146]	valid_0's bin

[370]	valid_0's binary_logloss: 1.42986
[371]	valid_0's binary_logloss: 1.43201
[372]	valid_0's binary_logloss: 1.43478
[373]	valid_0's binary_logloss: 1.43615
[374]	valid_0's binary_logloss: 1.44173
[375]	valid_0's binary_logloss: 1.44439
[376]	valid_0's binary_logloss: 1.44223
[377]	valid_0's binary_logloss: 1.44237
[378]	valid_0's binary_logloss: 1.44678
[379]	valid_0's binary_logloss: 1.44944
[380]	valid_0's binary_logloss: 1.44991
[381]	valid_0's binary_logloss: 1.44939
[382]	valid_0's binary_logloss: 1.45172
[383]	valid_0's binary_logloss: 1.45172
[384]	valid_0's binary_logloss: 1.45205
[385]	valid_0's binary_logloss: 1.45173
[386]	valid_0's binary_logloss: 1.45417
[387]	valid_0's binary_logloss: 1.45325
[388]	valid_0's binary_logloss: 1.45573
[389]	valid_0's binary_logloss: 1.4574
[390]	valid_0's binary_logloss: 1.4575
[391]	valid_0's binary_logloss: 1.45735
[392]	valid_0's binary_logloss: 1.45774
[393]	valid_0's binary_logloss: 1.45782
[394]	valid_0's binary_logloss: 1.45882
[3

...

[336]	valid_0's binary_logloss: 0.572509
[337]	valid_0's binary_logloss: 0.572518
[338]	valid_0's binary_logloss: 0.572449
[339]	valid_0's binary_logloss: 0.57254
[340]	valid_0's binary_logloss: 0.572553
[341]	valid_0's binary_logloss: 0.572693
[342]	valid_0's binary_logloss: 0.572864
[343]	valid_0's binary_logloss: 0.572911
[344]	valid_0's binary_logloss: 0.573131
[345]	valid_0's binary_logloss: 0.572875
[346]	valid_0's binary_logloss: 0.572934
[347]	valid_0's binary_logloss: 0.572793
[348]	valid_0's binary_logloss: 0.572965
[349]	valid_0's binary_logloss: 0.573102
[350]	valid_0's binary_logloss: 0.573327
[351]	valid_0's binary_logloss: 0.573077
[352]	valid_0's binary_logloss: 0.572982
[353]	valid_0's binary_logloss: 0.573237
[354]	valid_0's binary_logloss: 0.573282
[355]	valid_0's binary_logloss: 0.573422
[356]	valid_0's binary_logloss: 0.573404
[357]	valid_0's binary_logloss: 0.573295
[358]	valid_0's binary_logloss: 0.573356
[359]	valid_0's binary_logloss: 0.573612
[360]	valid_0's b

[624]	valid_0's binary_logloss: 0.580654
[625]	valid_0's binary_logloss: 0.580752
[626]	valid_0's binary_logloss: 0.580884
[627]	valid_0's binary_logloss: 0.580866
[628]	valid_0's binary_logloss: 0.580973
[629]	valid_0's binary_logloss: 0.580936
[630]	valid_0's binary_logloss: 0.580877
[631]	valid_0's binary_logloss: 0.580975
[632]	valid_0's binary_logloss: 0.581155
[633]	valid_0's binary_logloss: 0.581128
[634]	valid_0's binary_logloss: 0.581173
[635]	valid_0's binary_logloss: 0.581335
[636]	valid_0's binary_logloss: 0.581345
[637]	valid_0's binary_logloss: 0.58154
[638]	valid_0's binary_logloss: 0.581596
[639]	valid_0's binary_logloss: 0.581732
[640]	valid_0's binary_logloss: 0.582037
[641]	valid_0's binary_logloss: 0.581992
[642]	valid_0's binary_logloss: 0.582161
[643]	valid_0's binary_logloss: 0.582354
[644]	valid_0's binary_logloss: 0.582436
[645]	valid_0's binary_logloss: 0.582545
[646]	valid_0's binary_logloss: 0.58273
[647]	valid_0's binary_logloss: 0.582905
[648]	valid_0's bi

| 28        | 0.5556    | 0.01084   | 4.683     | 855.9     |
[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's binary_logloss: 0.614191
[2]	valid_0's binary_logloss: 0.607403
[3]	valid_0's binary_logloss: 0.601126
[4]	valid_0's binary_logloss: 0.596496
[5]	valid_0's binary_logloss: 0.591837
[6]	valid_0's binary_logloss: 0.589963
[7]	valid_0's binary_logloss: 0.585016
[8]	valid_0's binary_logloss: 0.582024
[9]	valid_0's binary_logloss: 0.577953
[10]	valid_0's binary_logloss: 0.578576
[11]	valid_0's binary_logloss: 0.580944
[12]	valid_0's binary_logloss: 0.575467
[13]	valid_0's binary_logloss: 0.576737
[14]	valid_0's binary_logloss: 0.578576
[15]	valid_0's binary_logloss: 0.57768
[16]	valid_0's binary_logloss: 0.582083
[17]	valid_0's binary_logloss: 0.584463
[18]	valid_0's binary_logloss: 0.584284
[19]	valid_0's binary_logloss: 0.58368
[20]	valid_0's binary_logloss: 0.585133
[21]	valid_0's binary_logloss: 0.588306
[22]	valid_0's binary_logloss: 0.585571
[23]	valid_0's bi

[396]	valid_0's binary_logloss: 0.99555
[397]	valid_0's binary_logloss: 0.995801
[398]	valid_0's binary_logloss: 0.995926
[399]	valid_0's binary_logloss: 0.999644
[400]	valid_0's binary_logloss: 0.999172
[401]	valid_0's binary_logloss: 1.00123
[402]	valid_0's binary_logloss: 1.00169
[403]	valid_0's binary_logloss: 1.00154
[404]	valid_0's binary_logloss: 1.00213
[405]	valid_0's binary_logloss: 1.00533
[406]	valid_0's binary_logloss: 1.00516
[407]	valid_0's binary_logloss: 1.00557
[408]	valid_0's binary_logloss: 1.00774
[409]	valid_0's binary_logloss: 1.00941
[410]	valid_0's binary_logloss: 1.01251
[411]	valid_0's binary_logloss: 1.01447
[412]	valid_0's binary_logloss: 1.01648
[413]	valid_0's binary_logloss: 1.01866
[414]	valid_0's binary_logloss: 1.02087
[415]	valid_0's binary_logloss: 1.02507
[416]	valid_0's binary_logloss: 1.02509
[417]	valid_0's binary_logloss: 1.0273
[418]	valid_0's binary_logloss: 1.02721
[419]	valid_0's binary_logloss: 1.02918
[420]	valid_0's binary_logloss: 1.031

[324]	valid_0's binary_logloss: 0.673046
[325]	valid_0's binary_logloss: 0.673749
[326]	valid_0's binary_logloss: 0.674151
[327]	valid_0's binary_logloss: 0.675673
[328]	valid_0's binary_logloss: 0.676132
[329]	valid_0's binary_logloss: 0.676481
[330]	valid_0's binary_logloss: 0.677112
[331]	valid_0's binary_logloss: 0.677536
[332]	valid_0's binary_logloss: 0.677728
| 30        | 0.5586    | 0.03207   | 6.263     | 332.2     |


In [35]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'n_estimators'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.012925143711798016, 'max_depth': 9.046486124621914, 'n_estimators': 339}
Best R-squared Score: 0.5893


In [36]:
params = {
    'max_depth': 9,            
    'learning_rate': 0.012925143711798016,  
    'n_estimators': 339
}

model_LGBM = LGBMClassifier(**params, random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM)
# предсказание ответов для тренеровочной выборки
y_pred_LGBM_train = model_LGBM.predict(x_train_LGBM)
# предсказание ответов для тестовой выборки
y_pred_LGBM = model_LGBM.predict(x_valid_LGBM)

print("Train LGBM f1_score: ", f1_score(y_train_LGBM, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train_LGBM, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid_LGBM, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid_LGBM, y_pred_LGBM))

Train LGBM f1_score:  0.8949416342412452
Train LGBM accuracy_score:  0.9304123711340206
Validation LGBM f1_score:  0.5892857142857143
Validation LGBM accuracy_score:  0.764102564102564


После подбора гиперпараметров мы улучшили метрики и снизили переобучение для модели LGBMClassifier на вадидации f1_score:  0.5892, accuracy_score:  0.764.

**xgboost**

In [37]:
def XGB_cv(max_depth, learning_rate, n_estimators):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = xg.XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        random_seed=27
    )
    model.fit(x_train_xgb, y_train_xgb)
    return f1_score(y_valid_xgb, model.predict(x_valid_xgb))

In [38]:
param_space = {
    'max_depth': (2, 10),            
    'learning_rate': (0.001, 0.3),  
    'n_estimators': (10, 2000)
}

bayesian_opt = BayesianOptimization(
    allow_duplicate_points=True,f=XGB_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=50)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:20:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 1         | 0.5439    | 0.1283    | 8.517     | 1.473e+03 |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:20:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 2         | 0.5641    | 0.2605    | 5.067     | 1.959e+03 |


...

| 53        | 0.5739    | 0.0836    | 6.92      | 136.5     |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:23:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 54        | 0.547     | 0.2765    | 4.87      | 1.959e+03 |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:23:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 55        | 0.4727    | 0.01312   | 2.343     | 697.7     |


In [39]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'n_estimators'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.21871539472326787, 'max_depth': 4.99449273109376, 'n_estimators': 1958}
Best R-squared Score: 0.5913


In [40]:
params = {
    'max_depth': 4,            
    'learning_rate':0.21871539472326787
    ,  
    'n_estimators': 1958    
}
model_XGB = xg.XGBClassifier(**params, random_state=27)
model_XGB.fit(x_train_xgb, y_train_xgb)
y_pred_xgb = model_XGB.predict(x_train_xgb)
y_val_xgb = model_XGB.predict(x_valid_xgb)
print("Train f1_score: ", f1_score(y_train_xgb, y_pred_xgb))
print("Train accuracy_score: ", accuracy_score(y_train_xgb, y_pred_xgb))
print("Validation f1_score: ", f1_score(y_valid_xgb, y_val_xgb))
print("Validation accuracy_score: ", accuracy_score(y_valid_xgb, y_val_xgb))

Train f1_score:  0.9157088122605364
Train accuracy_score:  0.9432989690721649
Validation f1_score:  0.5913043478260869
Validation accuracy_score:  0.7589743589743589


После подбора гиперпараметров мы улучшили метрики и снизили переобучение для модели XGBClassifier на вадидации f1_score: 0.5913, accuracy_score: 0.759.

**Выводы**

Мы сравнили 4 регрессионые модели: DecisionTreeRegressor, CatBoostRegressor, LGBMRegressor, XGBRegressor. Наилучшие результаты показали модели LGBMClassifier с результатом на трейне: f1_score= 0.89 и accuracy_score= 0.93  и тесте: f1_score=0.5892, accuracy_score=0.764 и XGBClassifier с результатом на трейне: f1_score= 0.92 и accuracy_score= 0.94  и тесте: f1_score=0.5913, accuracy_score=0.759.

**Рекомендации**

Для улучшения значения метрики и снижения переобучения нужно будет увеличить количество данных.